In [1]:
import os
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def sendNotification(app_name, message, additional_message='', event_name='program_log'):
    my_key = 'dtFZmutEg9ANUtSEpAU7Tu'
    data = {
        "value1" : app_name,
        "value2" : message,
        "value3" : additional_message
    }
    url = "https://maker.ifttt.com/trigger/" + event_name + "/with/key/" + my_key
    
    r = requests.post(url, data=data)
    
    return r

In [3]:
def waitForElement(driver, by_what=By.XPATH, element_info='', delay=20, do_quit=True):
    try:
        elem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((by_what, element_info)))
        print("Page is ready!")
        return elem
    except TimeoutException:
        print("Loading took too much time!")
        print("quiting while waiting for element:", element_info)
        if do_quit:
            report = "quiting while waiting for element: " + element_info
            send_report_and_close(report, driver)
            # driver.quit()
        return None

In [4]:
# sendNotification('Hello', 'Hello', event_name='program_log')

In [5]:
def send_report_and_close(report, driver):
    for i in range(10):
        r = sendNotification('Health report', report)
        if r.status_code == 200:
            break
        else:
            print("Hasn't send retrying... " + str(i) + " of 10")
  
    driver.close()
    exit()

In [6]:
report = ''
url = 'https://xmuxg.xmu.edu.cn/app/214'
path_to_driver = os.path.join(os.path.curdir, 'chromedriver')

driver = webdriver.Chrome(path_to_driver)
driver.get(url)


In [7]:
loaded_url = driver.current_url
if 'login' in loaded_url:
    # log in here then
    print('waiting to click')
    loging_button = waitForElement(driver, element_info="//button[contains(.,'统一身份认证')]")

    loging_button.click()


waiting to click
Page is ready!


In [8]:
    login_field = driver.find_element_by_xpath("//input[@id='username']")
    login_field.click()
    print('clicking login')
    # login_field.clear()
    login_field.send_keys('')

    password_field = driver.find_element_by_xpath("//input[@id='password']")
    password_field.click()
    print('clicking password')
    # password_field.clear()
    password_field.send_keys('')

clicking login
clicking password


In [9]:
    # press login/登录
    loging_button = driver.find_element_by_xpath("//button[contains(.,'登录/Login')]")
    loging_button.click()
    
    if loaded_url != driver.current_url:
        print("logged in successfully")
        report += 'Login OK.'
    else:
        print("Hasn't logged in")
        report = "Hasn't logged in. Check log:pass"
        send_report_and_close(report, driver)
        # TODO: send_report_and_close()

#
# /html/body/div[1]/div[@id='main']/div[@class='page-container container-fluid']/div[@class='page-content row']/div[@class='col-sm-12 page left-show']/div[@id='mainPage-page']/div[@class='main-p']/div[@class='shadow_box box_wrap_2']/div[2]/div[@class='box_main box_flex']/div[@class='app_child box_flex'][1]/div[@class='grow_1 box_flex column justify_center']/div[@class='title box_flex']

logged in successfully


In [10]:
dhr_section = waitForElement(driver, element_info="//div[contains(text(),'Daily Health Report')]")

Page is ready!


In [11]:
dhr_section.click()

### New window choosing

In [12]:
driver.switch_to.window(driver.window_handles[0])
driver.close()

driver.switch_to.window(driver.window_handles[-1])

In [13]:
# 健康打卡
menu_button = waitForElement(driver, element_info="//div[contains(@class, 'tab')][2]")

Page is ready!


In [14]:
# CHECK DATE
from datetime import datetime

curdate = datetime.today().strftime('%Y-%m-%d')

print(curdate)

2020-06-05


In [15]:
if len(driver.find_elements_by_xpath("//span[text()[contains(.,'" + curdate + "')]]")) > 0:
    print("date is correct:", curdate)
    report += "Date OK."
else:
    print("there's something wrong with date "+ curdate +", but we'll continue anyway")
    report += "Date BAD."

date is correct: 2020-06-05


In [16]:
# CHECK TIME
hours = int(datetime.today().strftime('%H'))
minutes = int(datetime.today().strftime('%M'))

if (hours >= 19 and minutes >= 30) or hours < 7:
    print("too late for the daily health report")
    report += "Time BAD."
    send_report_and_close(report, driver)
    # TODO: send_report_and_close()
else:
    print("time's alright:", hours, ':', minutes)
    report += "Time OK."

time's alright: 17 : 47


In [17]:
# Finally getting to the reporting part,
# menu_button = driver.find_element_by_xpath("//div[contains(@class, 'tab')][2]")
# menu_button = driver.find_element_by_xpath("//div[text()='我的菜单')]")
menu_button.click()


In [19]:
# confirmation = driver.find_element_by_css_selector('div.page-header-fixed.blue-style.reset-container div.page-container.container-fluid div.page-content.row div.col-sm-12.page.left-show div.app-detail-page-form:nth-child(2) div.middle.middle-top div.preview-container:nth-child(1) div.preview-page.pc-view div.container-fluid.form-preview-content.pc-view.form-view:nth-child(4) div.form-style div.row.cell-div-pc:nth-child(23) div.form-group.form-cell.col-sm-12 div.v-select.btn-block.info-value.btn-group > div.form-control.dropdown-toggle')
confirmation = driver.find_elements_by_xpath("//div[contains(@class, 'form-control dropdown-toggle')]")

In [20]:
# confirmation

In [21]:
save_button = driver.find_elements_by_xpath("//span[text()[contains(.,'保存')]]")[-1]

In [22]:
# sendNotification("DH", report)
# driver.close()
print("It should be done by now")
send_report_and_close(report, driver)

It should be done by now
